In [1]:
import numpy as np
import pandas as pd
import torch

from decifer import (
    DeciferDataset,
    Decifer,
    DeciferConfig,
    load_model_from_checkpoint,
    Tokenizer,
    extract_prompt,
    replace_symmetry_loop_with_P1,
    extract_space_group_symbol,
    reinstate_symmetry_loop,
)

In [2]:
# load a cif string and its xrd
dataset = DeciferDataset('../data/crystallm/5k/serialized/test.h5', ['cif_string', 'cif_tokenized', 'xrd_cont.iq', 'xrd_cl_emb'])
decode = Tokenizer().decode
padding_id = Tokenizer().padding_id
dataset_iter = iter(dataset)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = load_model_from_checkpoint('../cl_embeddings_experiment/ckpt.pt', device)
model.eval()

number of total parameters: 6.48M


Decifer(
  (transformer): ModuleDict(
    (cond_embedding): Identity()
    (wte): Embedding(372, 512)
    (wpe): Embedding(1024, 512)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-1): 2 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=512, out_features=1536, bias=False)
          (c_proj): Linear(in_features=512, out_features=512, bias=False)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=512, out_features=2048, bias=False)
          (c_proj): Linear(in_features=2048, out_features=512, bias=False)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=512, out_features=372, bias=False)
)

In [20]:
cif_string, cif_tokenized, xrd, cl_emb = next(dataset_iter)

prompt = extract_prompt(cif_tokenized, model.device, add_composition=False, add_spacegroup=False).unsqueeze(0)

if True:
    cond_vec = cl_emb.to(model.device).unsqueeze(0)
else:
    cond_vec = xrd.to(model.device).unsqueeze(0)
token_ids = model.generate_batched_reps(prompt, max_new_tokens=1000, cond_vec=cond_vec, start_indices_batch=[[0]]).cpu().numpy()
token_ids = [ids[ids != padding_id] for ids in token_ids]  # Remove padding tokens

out_cif = decode(list(token_ids[0]))
out_cif = replace_symmetry_loop_with_P1(out_cif)

# Extract space group symbol from the CIF string
spacegroup_symbol = extract_space_group_symbol(out_cif)

# If the space group is not "P1", reinstate symmetry
if spacegroup_symbol != "P 1":
    out_cif = reinstate_symmetry_loop(out_cif, spacegroup_symbol)
    
# print(cif_string)
# print()
print(out_cif)

Generating sequence:   0%|          | 0/1000 [00:00<?, ?it/s]

Exception: could not extract space group from:
_symmetry_equiv_pos_site_id
_symmetry_equiv_pos_as_xyz
1  'x, y, z'
d_symmetry_equiv_pos_site_id
_symmetry_equiv_pos_as_xyz
1  'x, y, z'
a_symmetry_equiv_pos_site_id
_symmetry_equiv_pos_as_xyz
1  'x, y, z'
t_symmetry_equiv_pos_site_id
_symmetry_equiv_pos_as_xyz
1  'x, y, z'
a_symmetry_equiv_pos_site_id
_symmetry_equiv_pos_as_xyz
1  'x, y, z'
__symmetry_equiv_pos_site_id
_symmetry_equiv_pos_as_xyz
1  'x, y, z'
A_symmetry_equiv_pos_site_id
_symmetry_equiv_pos_as_xyz
1  'x, y, z'
l_symmetry_equiv_pos_site_id
_symmetry_equiv_pos_as_xyz
1  'x, y, z'
6_symmetry_equiv_pos_site_id
_symmetry_equiv_pos_as_xyz
1  'x, y, z'
T_symmetry_equiv_pos_site_id
_symmetry_equiv_pos_as_xyz
1  'x, y, z'
i_symmetry_equiv_pos_site_id
_symmetry_equiv_pos_as_xyz
1  'x, y, z'
4_symmetry_equiv_pos_site_id
_symmetry_equiv_pos_as_xyz
1  'x, y, z'
L_symmetry_equiv_pos_site_id
_symmetry_equiv_pos_as_xyz
1  'x, y, z'
i_symmetry_equiv_pos_site_id
_symmetry_equiv_pos_as_xyz
1